In [1]:
import pandas as pd

df = pd.read_csv('/Users/annakelmanson/Desktop/lauzhack/NearestNeighbors/files/csv/BRISTOR_Zegoland_Activity.csv')

df.head()

,Country,Product,Data type,Channel,Date,Value
0,Zegoland,BRISTOR,Activity,Email,juil-20,29
1,Zegoland,BRISTOR,Activity,Email,août-20,146
2,Zegoland,BRISTOR,Activity,Email,sept-20,234
3,Zegoland,BRISTOR,Activity,Email,oct-20,1371
4,Zegoland,BRISTOR,Activity,Email,nov-20,1655


In [71]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [2]:

french_to_num = {
    'janv': '01', 'févr': '02', 'mars': '03', 'avr': '04',
    'mai': '05', 'juin': '06', 'juil': '07', 'août': '08',
    'sept': '09', 'oct': '10', 'nov': '11', 'déc': '12'
}

def convert_french_date(date_str):
    month, year = date_str.split('-')
    month = month.lower()
    year = '20' + year  
    month_num = french_to_num[month]
    return f"{year}-{month_num}-01"

df['Date'] = df['Date'].apply(convert_french_date)


In [3]:
df

,Country,Product,Data type,Channel,Date,Value
0,Zegoland,BRISTOR,Activity,Email,2020-07-01,29
1,Zegoland,BRISTOR,Activity,Email,2020-08-01,146
2,Zegoland,BRISTOR,Activity,Email,2020-09-01,234
3,Zegoland,BRISTOR,Activity,Email,2020-10-01,1371
4,Zegoland,BRISTOR,Activity,Email,2020-11-01,1655
...,...,...,...,...,...,...
251,Zegoland,BRISTOR,Activity,Telephone,2024-06-01,55
252,Zegoland,BRISTOR,Activity,Telephone,2024-07-01,44
253,Zegoland,BRISTOR,Activity,Telephone,2024-08-01,29
254,Zegoland,BRISTOR,Activity,Telephone,2024-09-01,55


In [5]:
import pandas as pd

In [53]:
df_original = pd.read_csv('/Users/annakelmanson/Desktop/lauzhack/NearestNeighbors/files/BRISTOR_Zegoland.csv')
TARGET = '{"Ex-factory volumes","BRISTOR"}'
ALL_FEATURES = [c for c in df_original.columns if c not in [TARGET, 'Date', 'Country']]
NUM_FEATURES = 2

# Scale the data

In [65]:
CURRENT_FEATURES = ALL_FEATURES[:NUM_FEATURES]
df = df_original[CURRENT_FEATURES + [TARGET]].rename(columns={TARGET: 'target'})
scaler = MinMaxScaler()
df[CURRENT_FEATURES + ['target']] = scaler.fit_transform(df[CURRENT_FEATURES + ['target']])
df.head()

,"{""Activity"",""Email"",""BRISTOR""}","{""Activity"",""Face to face call"",""BRISTOR""}",target
0,0.509642,0.000000,0.000000
1,0.463728,0.033079,0.044373
2,0.910009,0.043621,0.051346
3,0.667585,0.131225,0.056854
4,0.367309,0.234097,0.143671


# Train the model


In [73]:

# Create and train model
X = df[CURRENT_FEATURES].values
y = df['target'].values


§

In [75]:
X, y

(array([[0.50964187, 0.        ],
        [0.46372819, 0.03307888],
        [0.91000918, 0.0436205 ],
        [0.66758494, 0.13122501],
        [0.36730946, 0.23409669],
        [0.29109275, 0.30679753],
        [0.36730946, 0.28789531],
        [0.45821855, 0.35296256],
        [0.36547291, 0.28098873],
        [0.62993572, 0.34714649],
        [0.60422406, 0.22755362],
        [0.49770432, 0.34641948],
        [0.99816345, 0.49000364],
        [0.82644628, 0.52780807],
        [0.56565657, 0.45547074],
        [0.63177227, 0.46310433],
        [0.49586777, 0.46782988],
        [0.59871442, 0.36713922],
        [1.        , 0.46310433],
        [0.60146924, 0.60668848],
        [0.59320478, 0.47364595],
        [0.36271809, 0.80916031],
        [0.24517906, 0.43402399],
        [0.28374656, 0.62849873],
        [0.28191001, 0.66412214],
        [0.33241506, 0.84514722],
        [0.07988981, 0.58851327],
        [0.05693297, 0.56815703],
        [0.12855831, 0.956743  ],
        [0.196

# Reverse the scaling transformation

In [66]:
df_unscaled = pd.DataFrame(scaler.inverse_transform(df[CURRENT_FEATURES + ['target']]), 
                          columns=CURRENT_FEATURES + ['target'])
df_unscaled.head()

,"{""Activity"",""Email"",""BRISTOR""}","{""Activity"",""Face to face call"",""BRISTOR""}",target
0,647.0,38.0,17250.46
1,597.0,129.0,21334.34
2,1083.0,158.0,21976.04
3,819.0,399.0,22482.96
4,492.0,682.0,30473.16


In [46]:
TARGET = '{"Ex-factory volumes","BRISTOR"}'
WINDOW_SIZE = 3

In [47]:
from sklearn.preprocessing import MinMaxScaler

df[f'{TARGET}_EMA'] = df[TARGET].ewm(span=WINDOW_SIZE).sum()


In [48]:
df

,Date,Country,"{""Activity"",""Email"",""BRISTOR""}","{""Activity"",""Face to face call"",""BRISTOR""}","{""Activity"",""Mail"",""BRISTOR""}","{""Activity"",""Remote call"",""BRISTOR""}","{""Activity"",""Telephone"",""BRISTOR""}","{""Demand volumes"",""Indication A"",""BRISTOR""}","{""Demand volumes"",""Indication A"",""Competitors""}","{""Demand volumes"",""Indication B"",""BRISTOR""}",...,"{""Patient numbers"",""Competitors"",""Indication B"",""New Patient %""}","{""Patient numbers"",""BRISTOR"",""Indication B"",""Total patients #""}","{""Patient numbers"",""Competitors"",""Indication B"",""Total patients #""}","{""Patient numbers"",""BRISTOR"",""Indication B"",""Total patients %""}","{""Patient numbers"",""Competitors"",""Indication B"",""Total patients %""}","{""Share of voice"",""Indication A"",""BRISTOR""}","{""Share of voice"",""Indication A"",""Competitors""}","{""Share of voice"",""Indication B"",""BRISTOR""}","{""Share of voice"",""Indication B"",""Competitors""}","{""Ex-factory volumes"",""BRISTOR""}_EMA"
0,2021-02-01,Zegoland,647,38,1808,34,253,16786.000000,3.315930e+06,NaN,...,1.000,0.0,23684.0,0.000,1.000,0.104353,0.895647,0.000000,0.998000,17250.460000
1,2021-03-01,Zegoland,597,129,1976,50,259,21709.000000,3.641900e+06,NaN,...,0.999,0.0,24292.0,0.000,1.001,0.109600,0.890400,0.000000,1.000000,29959.570000
2,2021-04-01,Zegoland,1083,158,1557,42,259,23780.000000,3.551709e+06,NaN,...,1.000,0.0,24605.0,0.000,1.000,0.035310,0.964690,0.000000,1.001000,36955.825000
3,2021-05-01,Zegoland,819,399,1415,71,227,24780.000000,3.193625e+06,NaN,...,0.999,0.0,24883.0,0.000,1.000,0.089040,0.910960,0.000000,1.001000,40960.872500
4,2021-06-01,Zegoland,492,682,898,36,149,28959.000000,3.406375e+06,NaN,...,1.001,0.0,25176.0,0.000,1.001,0.086283,0.913717,0.000000,1.001000,50953.596250
5,2021-07-01,Zegoland,409,882,535,32,170,33293.000000,3.977538e+06,NaN,...,0.999,0.0,25713.0,0.000,1.000,0.116749,0.883251,0.014000,0.986000,54637.118125
6,2021-08-01,Zegoland,492,830,597,11,181,40416.000000,3.435839e+06,NaN,...,1.000,0.0,25920.0,0.000,1.001,0.124427,0.875573,0.013000,0.985000,65263.039062
7,2021-09-01,Zegoland,591,1009,697,20,173,42466.000000,3.535960e+06,NaN,...,1.001,0.0,26365.0,0.000,1.000,0.058884,0.941116,0.000000,0.998000,74608.359531
8,2021-10-01,Zegoland,490,811,399,9,183,48409.000000,3.687867e+06,NaN,...,1.000,0.0,26660.0,0.000,1.001,0.063087,0.936913,0.052000,0.946000,76887.639766
9,2021-11-01,Zegoland,778,993,960,21,237,50690.626885,3.614874e+06,2.000000,...,0.999,2.0,27188.0,0.000,0.999,0.064742,0.935258,0.094000,0.906000,94749.659883
